In [1]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "extractor"

In [2]:
from case.step_parser import parse_description

In [3]:
# Descriptions with actions
descriptions = [
    {"content": "Send 100 USDC to Bob", "action": "transfer"},
    {"content": "Transfer 0.005 ETH to Alice", "action": "transfer"},
    {"content": "Swap 1 USDC for DAI", "action": "swap"},
    {"content": "Stake 2 ETH in the Staking Pool", "action": "stake"},
    {"content": "Mint 1000 USDC", "action": "mint"},
    {"content": "Approve 100 USDC to Uniswap", "action": "approve"},
    {"content": "Borrow 100 USDC from Compound", "action": "borrow"},
    {"content": "Repay 0.5 ETH loan on Cream Finance", "action": "repay"},
    {"content": "Bridge 0.005 ETH from Ethereum to OP", "action": "bridge"},
    {"content": "Claim rewards from a liquidity mining pool", "action": "claim"},
    {"content": "Provide 1 ETH liquidity on SushiSwap", "action": "provide"},
    {"content": "Supply 1000 DAI to AAVE", "action": "supply"},
    {"content": "Mint a sports collectible NFT", "action": "mint"},
    {"content": "Withdraw 500 DAI from Compound", "action": "withdraw"},
    {"content": "Burn 1000 USDT from your wallet", "action": "burn"},
    {"content": "Sell a Monkey on OpenSea", "action": "sell"},
    {"content": "Buy a Pikachu on X", "action": "buy"},
]

In [4]:
import pandas as pd

actions = [
    {
        "description": desc["content"],
        " action": (parsed := parse_description(desc["content"])).action,
        "interact_with": parsed.interact_with,
    }
    for desc in descriptions
]

df = pd.DataFrame(actions)
df

,description,actual_action,interact_with
0,Send 100 USDC to Bob,transfer,USDC
1,Transfer 0.005 ETH to Alice,transfer,Alice
2,Swap 1 USDC for DAI,swap,USDC/DAI
3,Stake 2 ETH in the Staking Pool,stake,Staking Pool
4,Mint 1000 USDC,mint,USDC
5,Approve 100 USDC to Uniswap,approve,Uniswap
6,Borrow 100 USDC from Compound,borrow,Compound
7,Repay 0.5 ETH loan on Cream Finance,repay,Cream Finance
8,Bridge 0.005 ETH from Ethereum to OP,bridge,Ethereum to OP
9,Claim rewards from a liquidity mining pool,claim,liquidity mining pool


In [5]:
import csv

results = []
file_path = "data/raw/training/descriptions_action.csv"

with open(file_path) as file:
    reader = csv.DictReader(file)
    for row in reader:
        desc_content = row["description"]
        desc_action = row["action"]
        parsed = parse_description(desc_content)
        results.append(
            {
                "description": desc_content,
                "expected_action": desc_action,
                "actual_action": parsed.action,
                "is_action_correct": parsed.action == row["action"],
                "actual_interact_with": parsed.interact_with,
            }
        )

df_results = pd.DataFrame(results)
df_results

,description,expected_action,actual_action,is_action_correct,actual_interact_with
0,Swap 1 ETH for LINK,swap,swap,True,LINK
1,Approve 1000 USDT for Uniswap,approve,approve,True,Uniswap
2,Stake 0.5 ETH on Rocket Pool,stake,stake,True,Rocket Pool
3,Mint a CryptoPunk NFT,mint,mint,True,CryptoPunk contract
4,Transfer 200 USDC to Bob,transfer,transfer,True,USDC contract
...,...,...,...,...,...
295,Borrow 0.1 ETH from Compound using DAI as coll...,borrow,borrow,True,Compound
296,Repay 500 USDT loan on Yearn Finance,repay,repay,True,Yearn Finance
297,Withdraw 1000 DAI from Curve Finance pool,withdraw,withdraw,True,Curve Finance pool
298,Transfer 15 DAI to Alice,transfer,transfer,True,DAI


### Evaluation


In [6]:
correct_percentage = (df_results["is_action_correct"].sum() / len(df_results)) * 100
print(f"Correctness of extracting action: {correct_percentage:.2f}%")

# List of actions that are not correctly extracted
df_results[df_results["is_action_correct"] == False]

Correctness of extracting action: 99.00%


,description,expected_action,actual_action,is_action_correct,actual_interact_with
14,Send 5 LINK to Jane,send,transfer,False,LINK token contract
75,Send 100 USDT to a DAO,send,transfer,False,DAO
87,Transfer 15 USDC to Bob,send,transfer,False,USDC
